In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RYGate
from qiskit.circuit import Gate
from qiskit.quantum_info import Statevector

In [3]:
class SCS2Gate(Gate):
    def __init__(
        self,
        n: int,
    ) -> None:
        self.n = n
        super().__init__("SCS2", 2, [])

    def _define(self):
        qc = QuantumCircuit(2, name='SCS2')

        qc.cx(1, 0)

        theta = 2*np.arccos(np.sqrt(1/self.n))
        cry = RYGate(theta).control(ctrl_state="1")
        qc.append(cry, [0, 1])

        qc.cx(1, 0)

        self.definition = qc

class SCS3Gate(Gate):
    def __init__(
        self,
        l: int,
        n: int,
    ) -> None:
        self.l = l
        self.n = n
        super().__init__("SCS3", 3, [])

    def _define(self):
        qc = QuantumCircuit(3, name='SCS3')

        qc.cx(2, 0)

        theta = 2*np.arccos(np.sqrt(self.l/self.n))
        ccry = RYGate(theta).control(num_ctrl_qubits = 2, ctrl_state="11")
        qc.append(ccry, [0, 1, 2])

        qc.cx(2, 0)

        self.definition = qc

class SCSnkGate(Gate):
    def __init__(
        self,
        n: int,
        k: int,
    ) -> None:
        self.n = n
        self.k = k
        super().__init__(f"SCS{self.n},{self.k}", self.k+1, [])

    def _define(self):
        qc = QuantumCircuit(self.k+1, name=f"SCS{self.n},{self.k}")

        qc.append(SCS2Gate(n=self.n), [0, 1])

        for l in range(2, self.k+1):
            qc.append(SCS3Gate(l=l, n=self.n), [0, l-1, l])

        self.definition = qc

In [4]:
qc = QuantumCircuit(2)
qc.x(0)
qc.append(SCS2Gate(n=3), [0, 1])
qc.decompose().draw()

┌───────────┐┌───┐              ┌───┐
q_0: ┤ U3(π,0,π) ├┤ X ├──────■───────┤ X ├
     └───────────┘└─┬─┘┌─────┴──────┐└─┬─┘
q_1: ───────────────■──┤ Ry(1.9106) ├──■──
                       └────────────┘

In [5]:
Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

In [6]:
qc = QuantumCircuit(3)
qc.x(0)
qc.x(1)
qc.append(SCS3Gate(l=1, n=3), [0, 1, 2])
qc.decompose().draw()

┌───────────┐┌───┐              ┌───┐
q_0: ┤ U3(π,0,π) ├┤ X ├──────■───────┤ X ├
     ├───────────┤└─┬─┘      │       └─┬─┘
q_1: ┤ U3(π,0,π) ├──┼────────■─────────┼──
     └───────────┘  │  ┌─────┴──────┐  │  
q_2: ───────────────■──┤ Ry(1.9106) ├──■──
                       └────────────┘

In [7]:
Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

In [8]:
qc = QuantumCircuit(5)
qc.x(0)
qc.x(1)
qc.x(2)

qc.append(SCSnkGate(n=5, k=3), range(4))
qc.append(SCSnkGate(n=4, k=3), range(1, 5))
qc.append(SCSnkGate(n=3, k=2), range(2, 5))
qc.append(SCSnkGate(n=2, k=1), range(3, 5))
qc.decompose().decompose().draw()

┌──────────┐┌───┐              ┌───┐┌───┐              ┌───┐┌───┐»
q_0: ┤ U(π,0,π) ├┤ X ├──────■───────┤ X ├┤ X ├──────■───────┤ X ├┤ X ├»
     ├──────────┤└─┬─┘┌─────┴──────┐└─┬─┘└─┬─┘      │       └─┬─┘└─┬─┘»
q_1: ┤ U(π,0,π) ├──■──┤ Ry(2.2143) ├──■────┼────────■─────────┼────┼──»
     ├──────────┤     └────────────┘       │  ┌─────┴──────┐  │    │  »
q_2: ┤ U(π,0,π) ├──────────────────────────■──┤ Ry(1.7722) ├──■────┼──»
     └──────────┘                             └────────────┘       │  »
q_3: ──────────────────────────────────────────────────────────────■──»
                                                                      »
q_4: ─────────────────────────────────────────────────────────────────»
                                                                      »
«                        ┌───┐                                           »
«q_0: ──────■────────────┤ X ├───────────────────────────────────────────»
«           │       ┌───┐└─┬─┘            ┌───┐┌───┐           ┌───┐┌───┐»
«q_1: ──────┼───────┤ X ├──┼───────■──────┤ X ├┤ X ├─────■─────┤ X ├┤ X ├»
«           │       └─┬─┘  │  ┌────┴─────┐└─┬─┘└─┬─┘     │     └─┬─┘└─┬─┘»
«q_2: ──────■─────────■────┼──┤ Ry(2π/3) ├──■────┼───────■───────┼────┼──»
«     ┌─────┴──────┐       │  └──────────┘       │  ┌────┴────┐  │    │  »
«q_3: ┤ Ry(1.3694) ├───────■─────────────────────■──┤ Ry(π/2) ├──■────┼──»
«     └────────────┘                                └─────────┘       │  »
«q_4: ────────────────────────────────────────────────────────────────■──»
«                                                                        »
«                                                                         »
«q_0: ────────────────────────────────────────────────────────────────────»
«                     ┌───┐                                               »
«q_1: ─────■──────────┤ X ├───────────────────────────────────────────────»
«          │     ┌───┐└─┬─┘              ┌───┐┌───┐             ┌───┐     »
«q_2: ─────┼─────┤ X ├──┼────────■───────┤ X ├┤ X ├──────■──────┤ X ├─────»
«          │     └─┬─┘  │  ┌─────┴──────┐└─┬─┘└─┬─┘      │      └─┬─┘┌───┐»
«q_3: ─────■───────■────┼──┤ Ry(1.9106) ├──■────┼────────■────────┼──┤ X ├»
«     ┌────┴────┐       │  └────────────┘       │  ┌─────┴─────┐  │  └─┬─┘»
«q_4: ┤ Ry(π/3) ├───────■───────────────────────■──┤ Ry(1.231) ├──■────■──»
«     └─────────┘                                  └───────────┘          »
«                     
«q_0: ────────────────
«                     
«q_1: ────────────────
«                     
«q_2: ────────────────
«                ┌───┐
«q_3: ─────■─────┤ X ├
«     ┌────┴────┐└─┬─┘
«q_4: ┤ Ry(π/2) ├──■──
«     └─────────┘

In [9]:
qc.draw()

┌───┐┌─────────┐                                 
q_0: ┤ X ├┤0        ├─────────────────────────────────
     ├───┤│         │┌─────────┐                      
q_1: ┤ X ├┤1        ├┤0        ├──────────────────────
     ├───┤│  Scs5,3 ││         │┌─────────┐           
q_2: ┤ X ├┤2        ├┤1        ├┤0        ├───────────
     └───┘│         ││  Scs4,3 ││         │┌─────────┐
q_3: ─────┤3        ├┤2        ├┤1 Scs3,2 ├┤0        ├
          └─────────┘│         ││         ││  Scs2,1 │
q_4: ────────────────┤3        ├┤2        ├┤1        ├
                     └─────────┘└─────────┘└─────────┘

In [10]:
Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

In [11]:
n = 5
k = 3

for l in range(n, k, -1):
    print(l-k-1)
    print(l, k)
print()
for l in range(k, 1, -1):
    print(l, l-1)

1
5 3
0
4 3

3 2
2 1


In [12]:
n = 4
k = 3

qc = QuantumCircuit(5)
for i in range(k):
    qc.x(i)
    
for l in range(n, k, -1):
    qc.append(SCSnkGate(n=l, k=k), range(n-l, n-l+k+1))

for l in range(k, 1, -1):
    qc.append(SCSnkGate(n=l, k=l-1), range(n-l, n))

qc.decompose().decompose().draw()

┌──────────┐┌───┐            ┌───┐┌───┐           ┌───┐┌───┐           »
q_0: ┤ U(π,0,π) ├┤ X ├─────■──────┤ X ├┤ X ├─────■─────┤ X ├┤ X ├─────■─────»
     ├──────────┤└─┬─┘┌────┴─────┐└─┬─┘└─┬─┘     │     └─┬─┘└─┬─┘     │     »
q_1: ┤ U(π,0,π) ├──■──┤ Ry(2π/3) ├──■────┼───────■───────┼────┼───────┼─────»
     ├──────────┤     └──────────┘       │  ┌────┴────┐  │    │       │     »
q_2: ┤ U(π,0,π) ├────────────────────────■──┤ Ry(π/2) ├──■────┼───────■─────»
     └──────────┘                           └─────────┘       │  ┌────┴────┐»
q_3: ─────────────────────────────────────────────────────────■──┤ Ry(π/3) ├»
                                                                 └─────────┘»
q_4: ───────────────────────────────────────────────────────────────────────»
                                                                            »
«          ┌───┐                                                               
«q_0: ─────┤ X ├───────────────────────────────────────────────────────────────
«     ┌───┐└─┬─┘              ┌───┐┌───┐             ┌───┐                     
«q_1: ┤ X ├──┼────────■───────┤ X ├┤ X ├──────■──────┤ X ├─────────────────────
«     └─┬─┘  │  ┌─────┴──────┐└─┬─┘└─┬─┘      │      └─┬─┘┌───┐           ┌───┐
«q_2: ──■────┼──┤ Ry(1.9106) ├──■────┼────────■────────┼──┤ X ├─────■─────┤ X ├
«            │  └────────────┘       │  ┌─────┴─────┐  │  └─┬─┘┌────┴────┐└─┬─┘
«q_3: ───────■───────────────────────■──┤ Ry(1.231) ├──■────■──┤ Ry(π/2) ├──■──
«                                       └───────────┘          └─────────┘     
«q_4: ─────────────────────────────────────────────────────────────────────────
«

In [13]:
qc.draw()

┌───┐┌─────────┐                      
q_0: ┤ X ├┤0        ├──────────────────────
     ├───┤│         │┌─────────┐           
q_1: ┤ X ├┤1        ├┤0        ├───────────
     ├───┤│  Scs4,3 ││         │┌─────────┐
q_2: ┤ X ├┤2        ├┤1 Scs3,2 ├┤0        ├
     └───┘│         ││         ││  Scs2,1 │
q_3: ─────┤3        ├┤2        ├┤1        ├
          └─────────┘└─────────┘└─────────┘
q_4: ──────────────────────────────────────

In [14]:
from math import comb
comb(n, k)

4

In [15]:
Statevector(qc).draw('latex')

<IPython.core.display.Latex object>

In [19]:
class DickeStatePreparation(QuantumCircuit):
    def __init__(
        self,
        n,
        k
        ):
        self.n = n
        self.k = k
        
        qc = self.circuit()
        super().__init__(*qc.qregs, name=qc.name)
        self.compose(qc.to_gate(), qubits=self.qubits, inplace=True)
    
    def circuit(self):
        n = self.n
        k = self.k
        qc = QuantumCircuit(n, name=f"DickeStatePreparation {n},{k}")
        # for i in range(k):
        #     qc.x(i)
            
        for l in range(n, k, -1):
            qc.append(SCSnkGate(n=l, k=k), range(n-l, n-l+k+1))

        for l in range(k, 1, -1):
            qc.append(SCSnkGate(n=l, k=l-1), range(n-l, n))

        return qc

In [36]:
qc = QuantumCircuit(5)
qc.x(1)
qc.append(DickeStatePreparation(5, 3), range(5))
qc.draw()

┌────────────────────────────┐
q_0: ─────┤0                           ├
     ┌───┐│                            │
q_1: ┤ X ├┤1                           ├
     └───┘│                            │
q_2: ─────┤2 DickeStatePreparation 5,3 ├
          │                            │
q_3: ─────┤3                           ├
          │                            │
q_4: ─────┤4                           ├
          └────────────────────────────┘

In [35]:
Statevector(qc).draw('latex')

<IPython.core.display.Latex object>